# Hadoop Streaming assignment 2: Stop Words

Improve the previous program to calculate how many stop words are in the input dataset. Stop words list is in *‘/datasets/stop_words_en.txt’* file. Use Hadoop counter to count the number of stop words and total words in the dataset. The result is the percentage of stop words in the entire dataset (without percent symbol).

In [1]:
%%writefile stop_words_mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

stop_words = list()
words_count = 0
stop_words_count = 0

with open('stop_words_en.txt') as f1:
    stop_words = set(f1.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word in stop_words:
            stop_words_count += 1
        words_count += 1
        
print "%s\t%d" % ("total_words", words_count)
print "%s\t%d" % ("stop_words", stop_words_count)

Writing stop_words_mapper.py


In [2]:
%%writefile stop_words_reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print >> sys.stderr, "reporter:counter:wiki,%s,%d" % (current_key, word_sum)
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print >> sys.stderr, "reporter:counter:wiki,%s,%d" % (current_key, word_sum)
    print "%s\t%d" % (current_key, word_sum)

Writing stop_words_reducer.py


In [3]:
%%writefile get_stopword_percentage.py

import sys
import re

total_count = 0.0
stop_count = 0.0

for line in sys.stdin:
    try:
        key, count = line.strip().split('=', 1)
        count = int(count)
    except ValueError as e:
        continue
    if key == 'stop_words':
        stop_count += count
    if key == 'total_words':
        total_count += count

result = stop_count / total_count * 100
print "%5.3f" % result

Writing get_stopword_percentage.py


In [5]:
%%bash

OUT_DIR="stopwords_result_"$(date +"%s%6N")
NUM_REDUCERS=1
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopWords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files stop_words_mapper.py,stop_words_reducer.py,/datasets/stop_words_en.txt \
    -mapper "python stop_words_mapper.py" \
    -reducer "python stop_words_reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS
    
cat stderr_logs.txt | egrep "*_words" | python2 get_stopword_percentage.py
cat stderr_logs.txt >&2

38.440


rm: `stopwords_result_1533204505263416': No such file or directory
18/08/02 10:08:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/02 10:08:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/02 10:08:29 INFO mapred.FileInputFormat: Total input files to process : 1
18/08/02 10:08:29 INFO mapreduce.JobSubmitter: number of splits:2
18/08/02 10:08:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1533143790063_0004
18/08/02 10:08:29 INFO impl.YarnClientImpl: Submitted application application_1533143790063_0004
18/08/02 10:08:29 INFO mapreduce.Job: The url to track the job: http://6b428e5a5e59:8088/proxy/application_1533143790063_0004/
18/08/02 10:08:29 INFO mapreduce.Job: Running job: job_1533143790063_0004
18/08/02 10:08:36 INFO mapreduce.Job: Job job_1533143790063_0004 running in uber mode : false
18/08/02 10:08:36 INFO mapreduce.Job:  map 0% reduce 0%
18/08/02 10:08:46 INFO mapreduce.Job:  map 100% reduce 0%
18/08/02 10: